<h3>Introduction</h3>

The manuscript "**[Classification of X-ray images into COVID-19, pneumonia, and TB using cGAN and fine-tuned deep transfer learning models](https://link.springer.com/article/10.1007/s42600-021-00174-z)**" presents a novel approach to classifying chest X-ray images into three categories: COVID-19, pneumonia, and tuberculosis (TB). The study combines conditional Generative Adversarial Networks (cGAN) with fine-tuned deep transfer learning models to improve the accuracy and reliability of the classification process.

Here’s a brief overview of the methodology and findings from the study:
<ul>
<li>Conditional GAN (cGAN): The cGAN is used to generate synthetic X-ray images that can augment the training dataset, addressing the issue of limited labeled data.</li>

<li>Deep Transfer Learning Models: The research leverages pre-trained deep learning models fine-tuned on the augmented dataset. This helps in transferring learned features from large datasets to the specific task of X-ray image classification.</li>

<li>Classification Performance: The proposed method demonstrates high accuracy in distinguishing between COVID-19, pneumonia, and TB. This hybrid approach aims to overcome common challenges in medical image classification, such as data scarcity and class imbalance.</li>

<li>Practical Implications: The approach offers a promising tool for aiding in the rapid and accurate diagnosis of these respiratory conditions, which is crucial in clinical settings, especially during pandemics like COVID-19.</li>

<h3>Code for Conditional Generative Adversarial Networks cGAN</h3>

In [6]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model

ModuleNotFoundError: No module named 'keras.utils.vis_utils'

In [14]:
def define_discriminator(in_shape=(224,224,3), n_classes=6):
  in_label = Input(shape=(1,))
  li = Embedding(n_classes, 50)(in_label)
  n_nodes = in_shape[0] * in_shape[1]
  li = Dense(n_nodes)(li)
  li = Reshape((in_shape[0], in_shape[1], 1))(li)
  in_image = Input(shape=in_shape)
  merge = Concatenate()([in_image, li])
  fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
  fe = LeakyReLU(alpha=0.2)(fe)
  fe = Flatten()(fe)
  fe = Dropout(0.4)(fe)
  out_layer = Dense(1, activation='sigmoid')(fe)
  model = Model([in_image, in_label], out_layer)
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

def define_generator(latent_dim, n_classes=6):
  in_label = Input(shape=(1,))
  li = Embedding(n_classes, 50)(in_label)
  n_nodes = 56*56
  li = Dense(n_nodes)(li)
  li = Reshape((56, 56, 1))(li)
  in_lat = Input(shape=(latent_dim,))
  n_nodes = 128* 56 * 56
  gen = Dense(n_nodes)(in_lat)
  gen = LeakyReLU(alpha=0.2)(gen)
  gen = Reshape((56, 56, 128))(gen)
  merge = Concatenate()([gen, li])
  gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
  gen = LeakyReLU(alpha=0.2)(gen)
  gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
  gen = LeakyReLU(alpha=0.2)(gen)
  out_layer = Conv2D(3, (7,7), activation='tanh', padding='same') (gen)
  model = Model([in_lat, in_label], out_layer)
  return model

def define_gan(g_model, d_model):
  d_model.trainable = False
  gen_noise, gen_label = g_model.input
  gen_output = g_model.output
  gan_output = d_model([gen_output, gen_label])
  model = Model([gen_noise, gen_label], gan_output)
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

def scale(X):
  X = X.astype('float32')
  X = (X - 127.5) / 127.5
  return X

def load_real_samples():
  TRAINING_DIR = "Path to images"
  train_image_generator = ImageDataGenerator(rescale = 1.0/255.0)
  train_generator = train_image_generator.flow_from_directory(TRAINING_DIR,target_size=(224,224),class_mode='categorical',batch_size=32)
  return train_generator

def generate_real_samples(dataset, n_samples):
  n = len(dataset[0][0])
  labels =dataset[0][1]
  label = []
  for i in range(n): label.append(np.argmax(labels[i]))
  label = np.reshape(label,(n,1))
  dataset = dataset[0][0]
  ix = randint(0, n, n_samples)
  X = dataset[ix]
  y = ones((n_samples, 1))
  return [X, label], y

def generate_latent_points(latent_dim, n_samples, n_classes=6):
  x_input = randn(latent_dim * n_samples)
  z_input = x_input.reshape(n_samples, latent_dim)
  labels = randint(0, n_classes, n_samples)
  return [z_input, labels]

def generate_fake_samples(generator, latent_dim, n_samples):
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  images = generator.predict([z_input, labels_input])
  y = zeros((n_samples, 1))
  return [images, labels_input], y

def save_plot(examples, epoch, n=5):
  print(examples.shape)
  examples = (examples + 1) / 2.0
  for i in range(n * n):
    pyplot.subplot(n, n, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(examples[i])
  filename = 'path to store output image/generate d_plot_e%03d.png' % (epoch+1)
  pyplot.savefig(filename)
  pyplot.close()

def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=32):
  [X_real,labels_real], y_real = generate_real_samples(dataset, n_samples)
  [x_fake,labels], y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
  save_plot(x_fake, epoch)
  filename = 'path to store output image/generate r_model_%03d.h5' % (epoch+1)
  g_model.save(filename)

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=32):
  bat_per_epo = int(dataset.samples / n_batch)
  half_batch = int(n_batch)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
      d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
      [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
      [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
      y_gan = ones((n_batch, 1))
      g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    if (i+1) % 10 == 0:
      summarize_performance(i, g_model, d_model, dataset, latent_dim)
  g_model.save('path to save model')

In [ ]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim)

In [15]:
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot

In [16]:
def generate_latent_points(latent_dim, n_samples, n_classes=6):
  x_input = randn(latent_dim * n_samples)
  z_input = x_input.reshape(n_samples, latent_dim)
  labels = randint(0, n_classes, n_samples)
  return [z_input, labels]

def save_plot(examples, n):
  print(examples.shape)
  """for i in range(100):pyplot.axis("off") pyplot.imshow(examples[i],cmap="gray")
  filename = '/content/drive/My Drive/X-Ray_folders/6- Class/output/train/NORMAL/genereated_image_%03d' % (i+1)
  pyplot.savefig(filename) pyplot.close()"""
  for i in range(n * n):
    pyplot.subplot(n, n, 1 + i)
    pyplot.axis('off')
    pyplot.imshow(examples[i],cmap="gray")
  filename ='generated_image_000'
  pyplot.savefig(filename)
  pyplot.show()

In [ ]:
model = load_model('path to model/cgan_generator.h5',compile=False)
latent_points, labels = generate_latent_points(100,100)
labels = np.array([2]*100)
X = model.predict([latent_points, labels])
X = (X + 1) / 2.0
save_plot(X, 6)

<h3> Code for Deep Transfer Learning Model - ResNet-50 </h3>

In [17]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
print (tf. version )
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline
import cv2

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
TRAINING_DIR = 'path to training dataset'
TEST_DIR = 'path to testing dataset'
VAL_DIR = 'path to validation dataset'
classes = ["COVID-MEDIUM","COVID-MILD","COVID- SEVERE","NORMAL","PNEUMONIA","TB"]
INIT_LR = 1e-1
BS = 8
EPOCHS = 12

training_size = 0
test_size = 0
val_size = 0
for i in classes:
  training_size += len(glob(TRAINING_DIR+"/"+i+"/*"))
  test_size += len(glob(TEST_DIR+"/"+i+"/*"))
  val_size += len(glob(VAL_DIR+"/"+i+"/*"))
print(training_size,test_size,val_size)
COVID_MED_DIR = 'path to training dataset/COVID-MEDIUM'
COVID_MILD_DIR = 'path to training dataset/COVID-MILD'
COVID_SEVERE_DIR = 'path to training dataset/6- Class/COVID-SEVERE'
NORMAL_DIR = 'path to training dataset/6- Class/NORMAL'
PNEUMONIA_DIR = 'path to training dataset/6- Class/PNEUMONIA'
TB_DIR = 'path to training dataset/6-Class/TB'


covid_med_c = len(glob(COVID_MED_DIR+"/*")) + 100
covid_mild_c = len(glob(COVID_MILD_DIR+"/*")) + 100
covid_severe_c = len(glob(COVID_SEVERE_DIR+"/*")) + 100
normal_c = len(glob(NORMAL_DIR+"/*"))
pneumonia_c = len(glob(PNEUMONIA_DIR+"/*"))
tb_c = len(glob(TB_DIR+"/*"))
class_weight = [covid_med_c,covid_mild_c,covid_severe_c,normal_c, pneumonia_c,tb_c]
print(class_weight)

x = max(class_weight)
for i in range(0,len(class_weight)):
  class_weight[i] = round(x/class_weight[i],2)
print(class_weight)
class_weights = {}
for i in range(0,len(class_weight)):
  class_weights[i] = class_weight[i]
print(class_weights)

train_image_generator = ImageDataGenerator(rescale = 1.0/255.)
validation_image_generator = ImageDataGenerator(rescale = 1.0/255.)
train_generator = train_image_generator.flow_from_directory(TRAINING_DIR,target_size=(224,224),class_mode='categorical')
valid_generator = validation_image_generator.flow_from_directory( VAL_DIR,target_size=(224,224),class_mode="categorical")
test_generator = validation_image_generator.flow_from_directory(TEST_DIR,target_size=(224,224),class_mode="categorical")
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print("labels ", labels)
sample_training_data,sample_training_label = next(train_generator)

def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()
plotImages(sample_training_data[:5])
print(sample_training_label[:5])

base_model = keras.applications.ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3),pooling='avg')

for layer in base_model.layers:
  layer.trainable = False
base_model.summary()
model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.Dense(6, activation = "softmax"))
model.summary()
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=INIT_LR),loss="categorical_crossentropy", metrics= ["accuracy"])
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
history = model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS,callbacks = [early_stopping_cb,tensorboard_cb],class_weight=class_weights)
model.save("path to save model/DW6CLASSRESNET50.h5")

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs = len(history.epoch)
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

baseline_results = model.evaluate(test_generator, batch_size=BS, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)

nrows = 3
ncols = 4
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

num_predictions = 12
testData,testLabels = next(test_generator)
test_indices = np.random.choice(np.arange(0, len(testLabels)), size=(num_predictions,))
test_images = np.stack(([testData[i] for i in test_indices]))
test_labels = np.stack(([testLabels[i] for i in test_indices]))

predictions = model.predict(test_images)
print(np.argmax(predictions[0]))


for i in range(num_predictions):
  prediction = np.argmax(predictions[i])
  test_label = np.argmax(test_labels[i])
  image = (test_images[i] * 255).astype("uint8")
  if prediction == test_label:
    rgb_color = (0, 255, 0)
  else:
    rgb_color = (255, 0, 0)

  cv2.putText(image, str(labels[prediction]), (0, 18),cv2.FONT_HERSHEY_SIMPLEX,0.75,rgb_color, 1)

  sp = plt.subplot(nrows, ncols, i + 1, title="label: %s" % test_labels[i])
  sp.axis('Off')
  plt.imshow(image)

plt.show()

<h4> Similar to the ResNet-50 you can use the same implementation with minute changes to run other models like ResNet-101, Xception, DenseNet-169, DenseNet201. </h4>

<h3> Code for Lung Segmentation </h3>

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tqdm import tqdm
import os
from cv2 import imread, createCLAHE
import cv2
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateSchedule r

In [ ]:
def dice_coef(y_true, y_pred):
  y_true_f = keras.flatten(y_true)
  y_pred_f = keras.flatten(y_pred)
  intersection = keras.sum(y_true_f * y_pred_f)
  return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred): return -dice_coef(y_true, y_pred)

def unet(input_size=(256,256,1)):
  inputs = Input(input_size)

  conv1 = Conv2D(32, (3, 3), activation='relu', padding='same') (inputs)
  conv1 = Conv2D(32, (3, 3), activation='relu', padding='same') (conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

  conv2 = Conv2D(64, (3, 3), activation='relu', padding='same') (pool1)
  conv2 = Conv2D(64, (3, 3), activation='relu', padding='same') (conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

  conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

  conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

  conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
  conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
  conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

  up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
  conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
  conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

  up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
  conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
  conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

  up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
  conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
  conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

  up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
  conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
  conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

  conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
  return Model(inputs=[inputs], outputs=[conv10])


model = unet(input_size=(512,512,1))
model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef, 'binary_accuracy'])

model.summary()

model.load_weights("path to model folder/6- Class/cxr_reg_weights.best.hdf5")

import keras
TRAINING_DIR = '/content/drive/MyDrive/X-Ray_folders/6- Class/output/train'
TEST_DIR = '/content/drive/MyDrive/X-Ray_folders/6- Class/output/test'
VAL_DIR = '/content/drive/MyDrive/X-Ray_folders/6- Class/output/val'

arr = ["COVID-MEDIUM","COVID-MILD","COVID- SEVERE","NORMAL","PNEUMONIA","TB"]

for a in arr:
  print(TEST_DIR+"/"+a+"/")
  files=glob(TEST_DIR+"/"+a+"/"+"*")
  c = 0
  for j in files:
    image = tf.keras.preprocessing.image.load_img(j,target_size=( 512,512),color_mode="grayscale")
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = input_arr/255
    input_arr = np.array([input_arr])
    prediction = model.predict(input_arr)
    plt.show()
    res = np.squeeze(prediction)
    (h,w) = res.shape
    color = np.array([[[255]*(3)]*(w)]*h)
    image = tf.keras.preprocessing.image.load_img(j,target_size=(512,512))
    image = keras.preprocessing.image.img_to_array(image)
    for i in range(h-1):
      for j in range(w-1):
        if(round(res[i][j])==1):
          for k in range(3):
            color[i][j][k] = image[i][j][k]
        elif(round(res[i][j])==0):
          for k in range(3):
            color[i][j][k] = round(res[i][j])
    c+=1
  cv2.imwrite("path to output folder/6- Class/output1/test/"+a+"/"+str(c)+".jpg",color)

[Supplementary Information](https://www.editorialmanager.com/rbme/download.aspx?id=14019&guid=aea56d1c-3f99-4b8f-8c60-74c9b6d7f790&scheme=1)